![](CRISP_DM.png)

In [ ]:
#Importing require packages
import pandas as pd
import numpy as np
from datetime import datetime
# for plotting
%matplotlib inline
pd.set_option('display.max_rows', 500)

## 3.1 Data Preparation
* Main goal is to understand the final data structure
* Needs to be supportable each step by visual analytics

### 3.1.1 Johns Hopkins GITHUB csv data


In [ ]:
# define the path of dataset file and read it using pandas
data_path=('C:/Users/dhame/ds_covid-19/data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
pd_raw = pd.read_csv(data_path)
#look at data
pd_raw.head()

In [ ]:
#Set all date (starting from 4) columns as time_index
time_index=pd_raw.columns[4:]

In [ ]:
#Inserst time_index to our dataframe as column named 'date'
df_plot = pd.DataFrame({'date':time_index})
df_plot.head()

In [ ]:
#Look at first 5 countries in dataset
pd_raw['Country/Region'].head()

In [ ]:
#look at the data of germany for first 4 dates after summing of cases in relative state for Germany country
pd_raw[pd_raw['Country/Region']=='Germany'].iloc[:,4::].sum(axis=0)[0:4]

In [ ]:
#making of country_list of future use
country_list=['Italy','US','Spain','Germany','Korea, South'] 

In [ ]:
#Applying sum of infect cases for entire country_list
for each in country_list:
    df_plot[each]=np.array(pd_raw[pd_raw['Country/Region']==each].iloc[:,4::].sum(axis=0))

In [ ]:
#Plotting
df_plot.set_index('date').plot()

### 3.1.2 Data Type  for Date values in dataset
* Date can be stored with different data type, and it's easy to convert it into ISO format

In [ ]:
# convert date columns value to datetime
time_index=[datetime.strptime( each,"%m/%d/%y") for each in df_plot.date] 
# convert back to date ISO norm in the form of string
time_str=[each.strftime('%Y-%m-%d') for each in time_index] 

In [ ]:
# Set date column as index of dataframe
df_plot['date']=time_index
# check the data type of our column named date
type(df_plot['date'])

In [ ]:
df_plot.head()

In [ ]:
#save corrected above dataframe into our local path in CSV format
df_plot.to_csv('C:/Users/dhame/ds_covid-19/data/processed/COVID_small_flat_table.csv',sep=';',index=False)

In [ ]:
df_plot.head()

## 3.2 Relational data model 
* In the relational model of databases, a primary key is a specific choice of a minimal set of attributes (columns) that   uniquely specify a tuple (row) in a relation (table) (source: wiki)

* A primary key’s main features are:
    * 1. It must contain a unique value for each row of data.
    * 2. It cannot contain null values.

In [ ]:
# load file path where our dataset is located on local drive
data_path='C:/Users/dhame/ds_covid-19/data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
#Reading dataframe
pd_raw=pd.read_csv(data_path)
#look at dataframe's data
pd_raw.head()

In [ ]:
#Let's rename our Country/Region and Province/State column as country and state respectively
pd_data_base=pd_raw.rename(columns={'Country/Region':'country','Province/State':'state'})
#Place no where state data is not available: in state column
pd_data_base['state']=pd_data_base['state'].fillna('no') 

In [ ]:
#we don't want the data of lat and long so remove it
pd_data_base=pd_data_base.drop(['Lat','Long'],axis=1)
pd_data_base.head()

In [ ]:
# creating new dataframe by making changes of below
pd_relational_model=pd_data_base.set_index(['state','country']) \
                                .T                              \
                                .stack(level=[0,1])             \
                                .reset_index()                  \
                                .rename(columns={'level_0':'date',
                                                   0:'confirmed'},)
pd_relational_model.head()

In [ ]:
# Check data types of every columns
pd_relational_model.dtypes

In [ ]:
# set date column datatype as datetime64[ns]
pd_relational_model['date']=pd_relational_model.date.astype('datetime64[ns]')

In [ ]:
pd_relational_model.dtypes

In [ ]:
# set confirmed column datatype as integer
pd_relational_model.confirmed=pd_relational_model.confirmed.astype(int)
# saving dataframe to local drive in CSV format
pd_relational_model.to_csv('C:/Users/dhame/ds_covid-19/data/processed/COVID_relational_confirmed.csv',sep=';',index=False)
pd_relational_model.head()

## 3.3 Making relational data frame for Johns Hopkins dataset containing state information of US Country

In [ ]:
data_path='C:/Users/dhame/ds_covid-19/data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
pd_raw_US=pd.read_csv(data_path)
pd_raw_US.head()

In [ ]:
# dropping all non relevant columns 
pd_raw_US=pd_raw_US.drop(['UID', 'iso2', 'iso3', 'code3', 'Country_Region','FIPS', 'Admin2',  'Lat', 'Long_', 'Combined_Key'],axis=1)
# renaming the column name for state
pd_data_base_US=pd_raw_US.rename(columns={'Province_State':'state'}).copy()

In [ ]:
## repeating the process as earlier 
pd_relational_model_US=pd_data_base_US.set_index(['state']) \
                                .T                              \
                                .stack()             \
                                .reset_index()                  \
                                .rename(columns={'level_0':'date',
                                                   0:'confirmed'},
                                                  )

# set every value of column country to US
pd_relational_model_US['country']='US'
# convert to datetime
pd_relational_model_US['date']=[datetime.strptime( each,"%m/%d/%y") for each in pd_relational_model_US.date] 
pd_relational_model_US.head()

In [ ]:
# update the large relational file with data of US
pd_relational_model_all=pd_relational_model[pd_relational_model['country']!='US'].reset_index(drop=True)

In [ ]:
pd_relational_model_all=pd.concat([pd_relational_model_all,pd_relational_model_US],ignore_index=True)

In [ ]:
pd_relational_model_all.to_csv('C:/Users/dhame/ds_covid-19/data/processed/20200424_COVID_relational_confirmed.csv',sep=';',index=False)

In [ ]:
print(pd_relational_model_all[pd_relational_model_all['country']=='US'])

#### 3.4.1 Cross check on full sum US data

In [ ]:
cross_check_US=pd_relational_model_all[pd_relational_model_all['country']=='US']

In [ ]:
cross_check_US[['date','country','confirmed']].groupby(['date','country']).sum()